In [5]:
import cv2
import random
import mediapipe as mp
import time

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# Define gesture based on landmarks
def get_gesture(landmarks):
    finger_tips = [8, 12, 16, 20]  # Index, Middle, Ring, Pinky
    thumb_tip = 4
    finger_fold_status = []

    for tip in finger_tips:
        if landmarks[tip].y < landmarks[tip - 2].y:
            finger_fold_status.append(1)
        else:
            finger_fold_status.append(0)

    thumb_open = landmarks[thumb_tip].x > landmarks[thumb_tip - 1].x

    if sum(finger_fold_status) == 0 and not thumb_open:
        return "Rock"
    elif sum(finger_fold_status) == 4 and thumb_open:
        return "Paper"
    elif finger_fold_status[0] == 1 and finger_fold_status[1] == 1 and finger_fold_status[2] == 0 and finger_fold_status[3] == 0:
        return "Scissors"
    else:
        return "Unknown"

# Result logic
def get_result(player, computer):
    if player == computer:
        return "Tie"
    elif (player == "Rock" and computer == "Scissors") or \
         (player == "Scissors" and computer == "Paper") or \
         (player == "Paper" and computer == "Rock"):
        return "You Win!"
    else:
        return "Computer Wins!"

cap = cv2.VideoCapture(0)

last_action_time = 0
action_delay = 2  # seconds
show_result = False
result_text = ""
computer_choice = ""

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)

    gesture = "Waiting..."
    current_time = time.time()

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = hand_landmarks.landmark
            gesture = get_gesture(landmarks)

            cv2.putText(frame, f'You: {gesture}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            # Perform game logic if enough time passed
            if gesture != "Unknown" and current_time - last_action_time > action_delay:
                computer_choice = random.choice(["Rock", "Paper", "Scissors"])
                result_text = get_result(gesture, computer_choice)
                last_action_time = current_time
                show_result = True

    if show_result:
        cv2.putText(frame, f'Computer: {computer_choice}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, result_text, (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    cv2.putText(frame, "Press 'k' to exit", (10, 430), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 200), 1)

    cv2.imshow("Rock Paper Scissors", frame)
    if cv2.waitKey(1) & 0xFF == ord('k'):
        break

cap.release()
cv2.destroyAllWindows()
